# 深度学习的实用层面

## 1. 配置机器学习应用

### 1.1 训练集、开发集、测试集

如前所述，应用机器学习/深度学习是一个迭代的过程，需要设定层数、隐藏层神经元数、学习速率、激活函数等一系列超参。

不同领域（NLP，CV）的超参设定，可能会有一些经验性的法则在里面，但通常离开相应的领域，即使应用同样的模型，超参设定的经验性法则也会不再试用。超参的最终结果，可能取决于一大批因素，比如训练集的大小、特征的数量、CPU/GPU的设定等等。可以说，在超参的设定上，并没有银弹，不存在一上来就能准确找到超参的办法，只能通过跑出的模型进行测试，再进一步迭代。

而将数据集分为训练集、开发集、测试集三部分，将有助于更高效地完成整个迭代过程。

传统机器学习的时代，最佳实践通常会将数据集分为三部分，60%的训练集，20%的开发集（也称交叉验证集），20%的测试集。整个工作流程是，不断地在训练集上训练模型，用开发集来测试哪个模型的表现最好，最终在实验了足够多次得到了一个足够好的模型后，使用测试集的评分来作为最终模型预测结果的无偏估计。**当不需要报告最终的无偏估计时，也可以仅仅分为70%的训练集和30%的测试集（实际上是开发集，不过按照惯例这里会叫做测试集，不是很好的命名规范）**。

而在如今的大数据时代，比如说我们可以获得百万条甚至更多的数据，由于开发集和测试集仅仅用于评分，只要能保证评分的稳定性，实际上并不需要20%这么多的数据，可能有10000条这样的量级就足够了。

大数据时代的另一个趋势是，人们常常在分布不一致的训练集和测试集上进行模型训练。比如猫的图片分类器，训练集的数据可能是从网上爬取下来的，分辨率较高，取景也相对专业；而测试集的数据来自用户通过APP进行上传获取，相对比较模糊，拍摄不太讲究。这样这两部分数据的分布就会不一致。这里的指导原则是**至少确保开发集和测试集的数据来自同一分布**。另一方面，由于深度学习对数据量的要求非常高，业界的一个趋势确实也是，想尽一切办法创造更多的训练数据，即便训练集会因此来自不同的分布。